In [ ]:
!pip install h5py pyyaml #to download and save the model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Keras Libraries
import os
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from __future__ import absolute_import, division, print_function
import os

train_folder= '/content/drive/MyDrive/chest_xray/train'
val_folder = '/content/drive/MyDrive/chest_xray/val/'
test_folder = '/content/drive/MyDrive/chest_xray/test'

checkpoint_path = "/content/checkpoints/model.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                       save_weights_only = True,
                                       verbose=1)



# train 
os.listdir(train_folder)
train_n = train_folder+'NORMAL/'
train_p = train_folder+'PNEUMONIA/'

cnn = Sequential()

#Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))

#Pooling
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu"))

# 2nd Pooling layer
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# Flatten the layer
cnn.add(Flatten())

# Fully Connected Layers
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'sigmoid', units = 1))

# Compile the Neural network
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(train_folder,
                                           target_size = (64, 64),
                                           batch_size = 32,
                                           class_mode = 'binary')

validation_set = test_datagen.flow_from_directory(val_folder,
                                             target_size=(64, 64),
                                             batch_size=32,
                                             class_mode='binary')

test_set = test_datagen.flow_from_directory(test_folder,
                                       target_size = (64, 64),
                                       batch_size = 32,
                                       class_mode = 'binary')


cnn_model = cnn.fit(training_set,
                    steps_per_epoch = 160,
                    epochs = 5,
                    validation_data = validation_set,
                    validation_steps = 624)

test_accu = cnn.evaluate(test_set,steps=624)

print('The testing accuracy is :',test_accu[1]*100, '%')

Found 5106 images belonging to 2 classes.
Found 126 images belonging to 2 classes.
Found 625 images belonging to 2 classes.
Epoch 1/5
160/160 [==============================] - 328s 2s/step - loss: 0.4212 - accuracy: 0.8145 - val_loss: 0.2359 - val_accuracy: 0.9048
Epoch 2/5
160/160 [==============================] - 58s 365ms/step - loss: 0.2456 - accuracy: 0.8982
Epoch 3/5
160/160 [==============================] - 60s 374ms/step - loss: 0.2090 - accuracy: 0.9089
Epoch 4/5
160/160 [==============================] - 59s 370ms/step - loss: 0.2022 - accuracy: 0.9205
Epoch 5/5
624/624 [==============================] - 143s 228ms/step - loss: 0.4790 - accuracy: 0.8192
The testing accuracy is : 81.91999793052673 %


In [ ]:
f = open("model.tf", "w")
cnn.save_weights("model.HDF5", save_format="tf")
f.close()